In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import pycaret
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# df_07_08 = pd.read_csv('../nss_capstone/df_07_08.csv').drop(columns = 'Unnamed: 0').fillna('')
# df_07_08 = df_07_08.iloc[::-1]

In [3]:
# df_07_08.to_csv('../nss_capstone/df_07_08.csv')

In [4]:
# nba_full_odds_07_22 = pd.concat(
#     map(pd.read_csv,['../nss_capstone/df_21_22.csv',
#                      '../nss_capstone/df_20_21_covid.csv',
#                      '../nss_capstone/df_19_20_covid.csv',
#                      '../nss_capstone/df_18_19.csv',
#                      '../nss_capstone/df_17_18.csv',
#                      '../nss_capstone/df_16_17.csv',
#                      '../nss_capstone/df_15_16.csv',
#                      '../nss_capstone/df_14_15.csv',
#                      '../nss_capstone/df_13_14.csv',
#                      '../nss_capstone/df_12_13.csv',
#                      '../nss_capstone/df_11_12_lockout.csv',
#                      '../nss_capstone/df_10_11.csv',
#                      '../nss_capstone/df_09_10.csv',
#                      '../nss_capstone/df_08_09.csv',
#                      '../nss_capstone/df_07_08.csv']), ignore_index = True)          


In [5]:
#nba_full_odds_07_22.to_csv('../Capstone!/nba_full_odds_07_22.csv')

In [4]:
df = pd.read_csv('../nss_capstone/nba_full_odds_07_22.csv').drop(columns =  'Unnamed: 0').fillna('')

In [ ]:
df['W/L_bool'] = ''

for i in range(0, len(df), 2):
    if df.iloc[i]['Final'] > df.iloc[i+1]['Final']:
        df.at[i, 'W/L_bool'] = '1'
        df.at[i+1, 'W/L_bool'] = '0'
    else:
        df.at[i, 'W/L_bool'] = '0'
        df.at[i+1, 'W/L_bool'] = '1'

In [5]:
df.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Open Total,Close Total,Open Spread,Close Spread,W/L,W/L_bool
0,2022-06-16,528,H,Boston,22,17,27,24,90,2.5,4.0,-175.0,5.5,,,2.5,4.0,L,0
1,2022-06-16,527,V,Golden State,27,27,22,27,103,212.5,211.5,155.0,103.5,212.5,211.5,,,W,1
2,2022-06-13,526,H,Golden State,27,24,24,29,104,3.5,4.0,-165.0,107.5,,,3.5,4.0,W,1
3,2022-06-13,525,V,Boston,16,23,35,20,94,212.5,211.0,145.0,2.5,212.5,211.0,,,L,0
4,2022-06-10,524,H,Boston,28,26,24,19,97,2.5,4.0,-165.0,1,,,2.5,4.0,L,0


In [7]:
# create train df

df['Date'] = pd.to_datetime(df['Date'])

train_exclude = {'2021':['2021-10-19','2022-06-16'],
                  '2019':['2019-10-22','2020-03-11'],
                  '2018':['2018-10-16','2019-06-13']}
                                           
            
               
for year in train_exclude.keys():
    start_date = pd.to_datetime(train_exclude[year][0])
    end_date = pd.to_datetime(train_exclude[year][1])
    df = df[(df['Date'] < start_date) | (df['Date'] > end_date)]
    

In [8]:
df[['Date', 'VH', 'Team']]

,Date,VH,Team
1162,2022-12-09,H,GoldenState
1163,2022-12-09,V,Brooklyn
1164,2022-12-09,H,Miami
1165,2022-12-09,V,Toronto
1166,2022-12-09,H,Memphis
...,...,...,...
38311,2007-10-30,V,Houston
38312,2007-10-30,H,GoldenState
38313,2007-10-30,V,Utah
38314,2007-10-30,H,SanAntonio


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

test_exclude = {'2007':['2007-10-30', '2008-06-17'],
                 '2008':['2008-10-28', '2009-06-14'],
                 '2009':['2009-10-27', '2010-06-17'],
                 '2010':['2010-10-26', '2011-06-12'],
                 '2011':['2011-12-25', '2012-06-21'],
                 '2012':['2012-10-30', '2013-06-20'],
                 '2013':['2013-10-29', '2014-06-15'],
                 '2014':['2014-10-28', '2015-06-16'],
                 '2015':['2015-10-27', '2016-06-19'],
                 '2016':['2016-10-25', '2017-06-12'],
                 '2017':['2017-10-17', '2018-06-08'],
                 '2019':['2019-10-22', '2020-10-11'],
                 '2020': ['2020-07-30', '2020-10-11']} # this range was games played in bubble.}


for year in test_exclude.keys():
    start_date = pd.to_datetime(test_exclude[year][0])
    end_date = pd.to_datetime(test_exclude[year][1])
    if year == '2020':
        nba_odds_test = df[(df['Date'] < start_date) | (df['Date'] > end_date)]
    else:
        nba_odds_test = df[~((df['Date'] >= start_date) & (df['Date'] <= end_date))]

In [ ]:
    #     if year == '2020':
#         nba_odds_train = df[(df['Date'] < start_date) | (df['Date'] > end_date)]
#     else:
#         nba_odds_train = df[~((df['Date'] >= start_date) & (df['Date'] <= end_date))]
        
        
        # '2020': ['2020-10-11','2020-07-30'], # this range was games played in bubble.